In [1]:
import os
import pprint
import re
import time
from pathlib import Path
from pxr import Usd, UsdGeom
from pybg3 import pak, lsf, _pybg3

In [2]:
def checktime(msg, cb):
    t_start = time.time()
    rv = cb()
    t_end = time.time()
    print(f"{msg}: {t_end - t_start}")
    return rv

In [3]:
BG3_ROOT = Path(os.environ.get("BG3_DATA", os.path.expanduser("~/l/bg3/Data")))
GUSTAV = checktime("Gustav.pak", lambda: pak.PakFile(BG3_ROOT / "Gustav.pak"))
SHARED = checktime("Shared.pak", lambda: pak.PakFile(BG3_ROOT / "Shared.pak"))
ENGINE = checktime("Engine.pak", lambda: pak.PakFile(BG3_ROOT / "Engine.pak"))
MODELS = checktime("Models.pak", lambda: pak.PakFile(BG3_ROOT / "Models.pak"))

Gustav.pak: 0.06760311126708984
Shared.pak: 0.004001140594482422
Engine.pak: 0.00037026405334472656
Models.pak: 0.06220293045043945


In [4]:
data = GUSTAV.file_data("Generated/Public/GustavDev/Assets/HLOD/BGH_SteelWatchFoundry_B/HLOD_1_0_0_1.GR2")
granny = _pybg3._GrannyReader.from_data(data)

In [9]:
root = granny.root
for name in dir(root):
  print(f"{name}: {root.__getattr__(name)}")
print(dir(root.ArtToolInfo))
print(root.ArtToolInfo.ArtToolPointerSize)
print(root.ArtToolInfo.UpVector)

Animations: <pybg3._pybg3._GrannyPtrSpan object at 0x121fffeb0>
ArtToolInfo: <pybg3._pybg3._GrannyPtr object at 0x121fffeb0>
ExporterInfo: <pybg3._pybg3._GrannyPtr object at 0x121fffeb0>
ExtendedData: <pybg3._pybg3._GrannyPtr object at 0x121fffeb0>
FromFileName: D:\Jenkins\workspace\Repo\FB\Main_BuildServer_HLOD\FW4\Stable\LSProjects\Apps\Gustav\Data\Generated\Public\GustavDev\Assets\HLOD\BGH_SteelWatchFoundry_B\HLOD_1_0_0_1.fbx
Materials: <pybg3._pybg3._GrannyPtrSpan object at 0x121fffeb0>
Meshes: <pybg3._pybg3._GrannyPtrSpan object at 0x121fffeb0>
Models: <pybg3._pybg3._GrannyPtrSpan object at 0x121fffeb0>
Skeletons: <pybg3._pybg3._GrannyPtrSpan object at 0x121fffeb0>
Textures: <pybg3._pybg3._GrannyPtrSpan object at 0x121fffeb0>
TrackGroups: <pybg3._pybg3._GrannyPtrSpan object at 0x121fffeb0>
TriTopologies: <pybg3._pybg3._GrannyPtrSpan object at 0x121fffeb0>
VertexDatas: <pybg3._pybg3._GrannyPtrSpan object at 0x121fffeb0>
['ArtToolMajorRevision', 'ArtToolMinorRevision', 'ArtToolPoint